In [ ]:
import keras
import numpy as np
from matplotlib import pyplot

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def define_discriminator(in_shape=(28,28,1)):
    init = keras.initializers.RandomNormal(stddev=0.02)
    model = keras.models.Sequential()
    
    model.add(keras.layers.Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init,
    input_shape=in_shape))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.LeakyReLU(alpha=0.2))
    
    model.add(keras.layers.Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.LeakyReLU(alpha=0.2))
    
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1, activation='linear', kernel_initializer=init))
    
    model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.5))
    return model

In [ ]:
def define_generator(latent_dim):
    init = keras.initializers.RandomNormal(stddev=0.02)
    
    model = keras.models.Sequential()
    
    n_nodes = 256 * 7 * 7
    model.add(keras.layers.Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Reshape((7, 7, 256)))
    # upsample to 14x14
    
    model.add(keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',
    kernel_initializer=init))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    # upsample to 28x28
    
    model.add(keras.layers.Conv2DTranspose(64, (4,4), strides=(2,2), padding='same',
    kernel_initializer=init))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('relu'))
    # output 28x28x1
    
    model.add(keras.layers.Conv2D(1, (7,7), padding='same', kernel_initializer=init))
    model.add(keras.layers.Activation('tanh'))
    
    return model

In [ ]:
def define_gan(generator, discriminator):
    discriminator.trainable = False
    model = keras.models.Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.5))
    
    return model

In [ ]:
def load_real_samples():
    (trainX, _), (_, _) = keras.datasets.mnist.load_data()
    X = np.expand_dims(trainX, axis=-1)
    X = X.astype('float32')
    X = (X - 127.5) / 127.5
    
    return X

In [ ]:
def generate_real_samples(dataset, n_samples):
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = np.ones((n_samples, 1))
    return X, y

In [ ]:
def generate_latent_points(latent_dim, n_samples):
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    
    return x_input

In [ ]:
def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(x_input)
    y = np.zeros((n_samples, 1))
    
    return X, y

In [ ]:
def summarize_performance(step, g_model, latent_dim, n_samples=100):
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    X = (X + 1) / 2.0
    for i in range(10 * 10):
        pyplot.subplot(10, 10, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
    filename1 = 'generated_plot_%06d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    filename2 = 'model_%06d.h5' % (step+1)
    g_model.save(filename2)
    print('Saved %s and %s' % (filename1, filename2))

In [ ]:
def plot_history(d1_hist, d2_hist, g_hist):
    pyplot.plot(d1_hist, label='dloss1')
    pyplot.plot(d2_hist, label='dloss2')
    pyplot.plot(g_hist, label='gloss')
    pyplot.legend()
    filename = 'plot_line_plot_loss.png'
    pyplot.savefig(filename)
    pyplot.close()
    print('Saved %s' % (filename))

In [ ]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=64):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    n_steps = bat_per_epo * n_epochs
    half_batch = int(n_batch / 2)
    d1_hist, d2_hist, g_hist = list(), list(), list()
    
    for i in range(n_steps):
        X_real, y_real = generate_real_samples(dataset, half_batch)
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        
        d_loss1 = d_model.train_on_batch(X_real, y_real)
        d_loss2 = d_model.train_on_batch(X_fake, y_fake)
        
        z_input = generate_latent_points(latent_dim, n_batch)
        y_real2 = np.ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(z_input, y_real2)
        
        print('>%d, d1=%.3f, d2=%.3f g=%.3f' % (i+1, d_loss1, d_loss2, g_loss))
        
        d1_hist.append(d_loss1)
        d2_hist.append(d_loss2)
        g_hist.append(g_loss)
        
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model, latent_dim)
        plot_history(d1_hist, d2_hist, g_hist)

In [ ]:
latent_dim = 100
discriminator = define_discriminator()
generator = define_generator(latent_dim)
gan_model = define_gan(generator, discriminator)
dataset = load_real_samples()
print(dataset.shape)
train(generator, discriminator, gan_model, dataset, latent_dim)